# MongoDB
- https://www.mongodb.com/
- https://docs.mongodb.com/manual/tutorial/getting-started/
- https://www.w3schools.com/nodejs/nodejs_mongodb.asp
- NoSQL DB
- json-based document database (unlike traditional row/column model used by SQL)
- Rich JSON documents are the most natural productive way to work with data
- supports arrays and nested objects as values
- allows for flexible and dynamic schemas
- supports aggregations and modern use-cases such as geo-based search, graph search and text searh
- queries are themselves JSON; just know your JSON!

## Installing MongoDB
- https://docs.mongodb.com/manual/administration/install-community/
- E.g., on Mac, you can use brew to install Mongodb
    - https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/
    - $ brew install mongodb

## MongoDB GUI-based management tools
- Robo 3T: free and opensource: https://www.robomongo.org/
- MongoDB Compass Community Edition: https://www.mongodb.com/download-center/compass
- There're web-based clients as well

## Run MongoDB server
- https://docs.mongodb.com/guides/server/install/
- add /bin folder to path or simply CD into the folder to run mongod app
- Using terminal: 
    - $ mongod --dbpath=[dbfolder]
    
- CD into MongoDB folder and run the following command:
    - $ mongod --dbpath=data
- MongoDB by defualt runs on localhost:27017

## Connect to the local/remote server
- you can use CLI mongo client or Compass
<img src="./resources/MongoDBCompass.png">
<img src="./resources/Robo3T.png">

## Connect to the server using NodeJS mongodb client
- install mongodb client driver for nodejs
```bash
$ npm install -g mongodb
```

In [15]:
var mongo = require("mongodb")

var MongoClient = require("mongodb").MongoClient
var url = "mongodb://localhost:27017/test"

MongoClient.connect(
    url,
    function(err, db) {
        if (err) throw err
        console.log("Database created!")
        // database is not actually created until one collection/table
        db.close()
    }
)

Database created!


## MongoDB CRUD Operations
- create
- read
- update
- delete

## create collection

```javascriopt
db.createCollection(collectionName, ()=>{})
```

In [16]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

// connect(url, settings, callback)
MongoClient.connect(url, {useUnifiedTopology: true}, function(err, db) {
  if (err) throw err;
  var dbo = db.db("test");
  dbo.createCollection("inventory", function(err, res) {
    if (err) throw err;
    console.log("Collection created!");
    db.close();
  });
});

Collection created!


## insert a single document
```javascript
dbo.insertOne( {key: value, key1: value1, ...}, ()=>{})
```

In [17]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
  if (err) throw err;
  var dbo = db.db("test");
  dbo.collection("inventory").insertOne({
    item: 'canvas',
      qty: 100,
      tags: ['cotton'],
      size: {h: 28, w:35.5, uom: 'cm'}
  })
    db.close();
  
});

## insert many documents
```javascript
dbo.insertMany([ json_doc1, json_doc2, ... ], ()=> {})
```

In [18]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    insertManyInventory(dbo);
    db.close();
});

function insertManyInventory(db) {
    db.collection('inventory').insertMany([
      {
        item: 'journal',
        qty: 25,
        tags: ['blank', 'red'],
        size: { h: 14, w: 21, uom: 'cm' }
      },
      {
        item: 'mat',
        qty: 85,
        tags: ['gray'],
        size: { h: 27.9, w: 35.5, uom: 'cm' }
      },
      {
        item: 'mousepad',
        qty: 25,
        tags: ['gel', 'blue'],
        size: { h: 19, w: 22.85, uom: 'cm' }
      }
    ], function(err, res) {
        console.log(`${res.insertedCount} document(s) inserted!`)
    });
}

3 document(s) inserted!


## select all documents in a collection
- SQL: select * from inventory
- must convert find all result to array
- use empty query object {}
```javascript
dbo.find({}).toArray(()=>{})
```

In [19]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    dbo.collection('inventory').find({}).toArray(function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

[ { _id: 5e9f525def0f7b825e9ddcaa,
    item: 'canvas',
    qty: 100,
    tags: [ 'cotton' ],
    size: { h: 28, w: 35.5, uom: 'cm' } },
  { _id: 5e9f525eef0f7b825e9ddcab,
    item: 'journal',
    qty: 25,
    tags: [ 'blank', 'red' ],
    size: { h: 14, w: 21, uom: 'cm' } },
  { _id: 5e9f525eef0f7b825e9ddcac,
    item: 'mat',
    qty: 85,
    tags: [ 'gray' ],
    size: { h: 27.9, w: 35.5, uom: 'cm' } },
  { _id: 5e9f525eef0f7b825e9ddcad,
    item: 'mousepad',
    qty: 25,
    tags: [ 'gel', 'blue' ],
    size: { h: 19, w: 22.85, uom: 'cm' } } ]


## seelct one document - find one document
- return the first document from results
```javascript
dbo.findOne({}, ()=>{})
```

In [20]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    dbo.collection('inventory').findOne({}, function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

{ _id: 5e9f525def0f7b825e9ddcaa,
  item: 'canvas',
  qty: 100,
  tags: [ 'cotton' ],
  size: { h: 28, w: 35.5, uom: 'cm' } }


## projection - limit fields/attributes to return from a query
- https://docs.mongodb.com/manual/tutorial/project-fields-from-query-results/
- {field: 0/1} : 0 omit, 1 include in result
- doesn't work here, but works fine in Compass!
```javascript
dbo.findOne({}, {field:0, field1:1, ...}, ()=>{})
```

In [22]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    // {fieldname: 1, fieldname: 0}; 1 include (bydefault), 0 exclude
    dbo.collection("inventory").findOne({}, {_id:0}, function (err, result) {
        if (err) throw err;
        console.log(result);
    });
    db.close();
});

{ _id: 5e9f525def0f7b825e9ddcaa,
  item: 'canvas',
  qty: 100,
  tags: [ 'cotton' ],
  size: { h: 28, w: 35.5, uom: 'cm' } }


## filter the result using query object
- SQL: select * from inventory where item='canvas'
- explictly create query object and pass it to find()
- must convert find result to array
```javascript
{fieldName: value}
```
- filter and return all documents having a given fieldName = value

In [23]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = {item: 'mat'};
    dbo.collection('inventory').find(query).toArray(function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

[ { _id: 5e9f525eef0f7b825e9ddcac,
    item: 'mat',
    qty: 85,
    tags: [ 'gray' ],
    size: { h: 27.9, w: 35.5, uom: 'cm' } } ]


## filter with regular expression
 - find only the documents/objects where the item starts with the letter "m" - /^m/
 ```javascript
 db.find({ fieldName: re })
 ```

In [24]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = {item: /^m/ };
    dbo.collection('inventory').find(query).toArray(function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

[ { _id: 5e9f525eef0f7b825e9ddcac,
    item: 'mat',
    qty: 85,
    tags: [ 'gray' ],
    size: { h: 27.9, w: 35.5, uom: 'cm' } },
  { _id: 5e9f525eef0f7b825e9ddcad,
    item: 'mousepad',
    qty: 25,
    tags: [ 'gel', 'blue' ],
    size: { h: 19, w: 22.85, uom: 'cm' } } ]


## specify AND, OR conditions and comparisons
- SQL: select * from inventory where item like 'm%' and qty >= 85
- Mongodb provides named operators for comparisons
- $gt:  >

- $gte: >= 

- $lt: <

- $lte: <=

- $ne: !=

```javascript
{ fieldName: {$gt: value} }
{ fieldName: {$lte: value} }
$or : [{fieldName1 : value1}, {fieldName2: value2}]
// and
{ fieldName1 : value1, fieldName2: {$lte: value2} }
```



In [25]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = {item: /^m/,
                qty: {$ne: 85}}; // AND
    dbo.collection('inventory').find(query).toArray(function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

[ { _id: 5e9f525eef0f7b825e9ddcad,
    item: 'mousepad',
    qty: 25,
    tags: [ 'gel', 'blue' ],
    size: { h: 19, w: 22.85, uom: 'cm' } } ]


In [26]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = {$or: [{item: /^m/}, {item: /^c/}], // OR
                 qty: {$gte: 85}}; // AND
    dbo.collection('inventory').find(query).toArray(function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

[ { _id: 5e9f525def0f7b825e9ddcaa,
    item: 'canvas',
    qty: 100,
    tags: [ 'cotton' ],
    size: { h: 28, w: 35.5, uom: 'cm' } },
  { _id: 5e9f525eef0f7b825e9ddcac,
    item: 'mat',
    qty: 85,
    tags: [ 'gray' ],
    size: { h: 27.9, w: 35.5, uom: 'cm' } } ]


## sort the result - non-decreasing order
- sort in ascending order (non-decreasing order) based on fieldName
```javascript
dbo.find({}).sort({ fieldName: 1 })
```

In [27]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = {$or: [{item: /^m/}, {item: /^c/}], // OR
                 qty: {$gte: 85}}; // AND
    dbo.collection('inventory').find(query).sort({qty: 1}).toArray(function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

[ { _id: 5e9f525eef0f7b825e9ddcac,
    item: 'mat',
    qty: 85,
    tags: [ 'gray' ],
    size: { h: 27.9, w: 35.5, uom: 'cm' } },
  { _id: 5e9f525def0f7b825e9ddcaa,
    item: 'canvas',
    qty: 100,
    tags: [ 'cotton' ],
    size: { h: 28, w: 35.5, uom: 'cm' } } ]


## sort the result - non-increasing order
- sort the result in descending (non-increasing) order
```javascript
dbo.find({}).sort({ fieldName: -1 })
```

In [28]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = {$or: [{item: /^m/}, {item: /^c/}], // OR
                 qty: {$gte: 85}}; // AND
    dbo.collection('inventory').find(query).sort({qty: -1}).toArray(function (err, result) {
        if (err) throw err;
        console.log(result)
    });
    db.close();
});

[ { _id: 5e9f525def0f7b825e9ddcaa,
    item: 'canvas',
    qty: 100,
    tags: [ 'cotton' ],
    size: { h: 28, w: 35.5, uom: 'cm' } },
  { _id: 5e9f525eef0f7b825e9ddcac,
    item: 'mat',
    qty: 85,
    tags: [ 'gray' ],
    size: { h: 27.9, w: 35.5, uom: 'cm' } } ]


## update document

### updateOne(query, newValue, function)

- use $set operator to set {key : newvalue}

```javascript
db.updateOne( {query}, {$set: {key : newvalue} }, ()=>{} )
```
- only 1 document (the first matching) is updated even if the query matches many documents

In [8]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = { item: /^m/,
                 qty: {$gte: 85}};
    var newValue = {$set: {qty: 75}}
    dbo.collection('inventory').updateOne(query, newValue, function (err, res) {
        if (err) throw err;
        console.log(`${res.result.nModified} document updated!`);
    });
    db.close();
});

1 document updated!


## updateMany(query, newValue, function) 
- update all documents that meet the criteria of the query
- use $set operator to set the { key : newvalue}

```javascript
db.updateMany({query}, {$set: {key: newValue})
```

In [9]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = { item: /^m/,
                 qty: {$gte: 85}};
    var newValue = {$set: {qty: 70}}
    dbo.collection('inventory').updateMany(query, newValue, function (err, res) {
        if (err) throw err;
        console.log(`${res.result.nModified} document(s) updated!`);
    });
    db.close();
});

0 document(s) updated!


## delete document
### deleteOne(query, function)
- delete a record/document (if query matches multiple documents, first occurance is deleted)
```javascript
db.deleteOne({query}, ( )={})
```

In [10]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = { item: /^m/,
                 qty: {$gte: 70}};
    dbo.collection('inventory').deleteOne(query, function (err, res) {
        if (err) throw err;
        console.log(`${res.result.n} document deleted!`);
    });
    db.close();
});

1 document deleted!


### deleteMany(query, function)
- delete all records/documents matching query
- must be careful with this; there's no undo!

```javascript
db.deleteMany({query}, ()=>{})
```

In [11]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    var query = { item: /^m/,
                 qty: 25};
    dbo.collection('inventory').deleteMany(query, function (err, res) {
        if (err) throw err;
        console.log(`${res.result.n} document(s) deleted!`);
    });
    db.close();
});

1 document(s) deleted!


## drop collection
- drop table/collection
```javascript
db.dropCollection(collectionName, ()=>{})
```

In [12]:
var MongoClient = require('mongodb').MongoClient;
var url = "mongodb://localhost:27017/";

MongoClient.connect(url, function(err, db) {
    if (err) throw err;
    var dbo = db.db("test");
    dbo.dropCollection('inventory', function (err, res) {
        if (err) throw err;
        console.log(`${res} Collection dropped!`);
    });
    db.close();
});

true Collection dropped!
